In [1]:
import torch
from supervoice_gpt import SupervoiceGPT, Tokenizer, config

In [2]:
# Model
tokenizer = Tokenizer(config, "tokenizer_text.model")
model = SupervoiceGPT(config).to("cpu")
# checkpoint = torch.load(f'./output/pitch-6.pt', map_location="cpu")
# model.load_state_dict(checkpoint['model'])
model.eval()
print('ok')

ok


In [3]:
torch.cuda.memory._record_memory_history()
tokens = model.generate("He that is giddy thinks the world turns round.", tokenizer, max_new_tokens = 256, top_k = 6, device = "cpu")
torch.cuda.memory._dump_snapshot("my_snapshot.pickle")
print(tokens)

[W unwind.cpp:194] Warning: Unsupported unwinding pattern: Address not in range (function unwinderFor)


[('vʲ', 80, 255), ('ɚ', 66, 140), ('sʲ', 66, 203), ('sʲ', 66, 157), ('ɒː', 46, 43), ('vʲ', 60, 107), ('vʲ', 83, 12), ('ʎ', 83, 43), ('vʲ', 74, 203), ('ɐ', 74, 203), ('ɐ', 66, 43), ('ɐ', 74, 157), ('ʎ', 46, 12), ('ɐ', 46, 43), ('ʎ', 34, 12), ('vʲ', 46, 27), ('sʲ', 74, 157), ('ɒː', 34, 175), ('ɒː', 74, 12), ('sʲ', 34, 43), ('vʲ', 74, 12), ('vʲ', 74, 12), ('ɝ', 74, 157), ('ʎ', 34, 203), ('vʲ', 34, 12), ('ʎ', 34, 157), ('r', 60, 157), ('vʲ', 60, 12), ('r', 60, 203), ('ʎ', 34, 175), ('ʎ', 60, 175), ('ʎ', 66, 12), ('r', 34, 12), ('sʲ', 66, 12), ('r', 46, 203), ('tsʲ', 60, 203), ('ʎ', 60, 157), ('tsʲ', 60, 11), ('ɐ', 46, 175), ('r', 34, 12), ('ʎ', 52, 146), ('ʎ', 34, 203), ('ɐ', 34, 11), ('tsʲ', 46, 12), ('ɐ', 46, 175), ('vʲ', 52, 157), ('r', 46, 175), ('ʎ', 52, 12), ('vʲ', 46, 12), ('r', 60, 12), ('sʲ', 52, 203), ('tsʲ', 74, 12), ('ʎ', 74, 157), ('vʲ', 46, 157), ('ʎ', 60, 43), ('tsʲ', 60, 12), ('r', 83, 175), ('tsʲ', 46, 175), ('sʲ', 60, 203), ('sʲ', 46, 203), ('tsʲ', 34, 12), ('vʲ', 66, 175

In [4]:
# input = "Hey, you?"
# ctx_tokens = ['ç', 'ɪ']
# ctx_durations = [8, 10]
# probs, p = model.predict_next(input, ctx_tokens, ctx_durations, tokenizer)
# for i in range(len(probs)):
#     print(p[i] + ": "+ str(probs[i]))
